# README
This document explains the theory to solve once instance of the problem. Most of the code is implemented in `TwentyFourSevenSolver.py`.

To solve the full puzzle (i.e. all four squares), run `main.py`. 

#### Load CP-SAT and sample data (top left grid)

In [ ]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()

import data
vals_list = data.vals_list_1
views = data.views_1

#### Problem domain
The problem is set on a $7 \times 7$ grid. We will use row-major one-dimensional indexing to label the cells.

The values to be filled in are 1 to 7, with 0 being used to represent an empty cell.

We will also need to track adjacencies of cells. For right-adjacencies, the adjacency matrix is $N \times N-1$. For down-adjacencies, it is $N-1 \times N$.

In [ ]:
N = 7 # size of grid

def n2ij(n, N=N):
    i = n // N
    j = n - i*N
    return (i,j)

def ij2n(i, j, N=N):
    return i*N + j
    
from itertools import product

nrange = [n for n in range(N**2)] # (0 to N^2-1)
vrange = [v for v in range(8)] # (0 to 7)
erange = list(product(range(N), range(N-1))) # (0 to N-1) x (0 to N-2)

#### Setup variables:
* $x_{n,v} = 1$ $\Leftrightarrow$ cell $(n)$ contains value $(v)$.

* $y_{n_1, n_2} = 1$ $\Leftrightarrow$ non-empty cells $(n_1)$ and $(n_2)$ are adjacent.

In [ ]:
x = {}
for n in nrange:
    for v in vrange:
        x[n,v] = model.NewBoolVar(f'x[{n},{v}]')
y = {}
for i,j in erange: 
    nl = ij2n(i  , j  )
    nr = ij2n(i  , j+1)
    nu = ij2n(j  , i  )
    nd = ij2n(j+1, i  )
    y[nl,nr] = model.NewBoolVar(f'y[{nl},{nr}]')
    y[nu,nd] = model.NewBoolVar(f'y[{nu},{nd}]')

#### Easy constraints on $x$ (refer to `Hooks8` for more details)
* Each cell $(n)$ should only be associated with one $(v)$.
* Each positive $(v)$ should appear $(v)$ times.
* There should be 4 positive $(v)$ in each row and each column.
* The sum of all $(v)$ on each row and column should be 20.
* For each $2 \times 2$ sub-matrix, check that the total number non-zero values is no more than 3.

In [ ]:
# each (n) has exactly one (v) (including 0)
for n in nrange:
    model.AddExactlyOne(x[n,v] for v in vrange)

# each (v>0) allocated (v) times
for v in vrange:
    if v > 0:
        model.Add(sum(x[n,v] for n in nrange) == v)

# four (v>0) per row/col, summing to 20
for i in range(N):
    model.Add(sum(  x[ij2n(i,j),v] for v in vrange for j in range(N) if v > 0) == 4)
    model.Add(sum(v*x[ij2n(i,j),v] for v in vrange for j in range(N)) == 20)
    model.Add(sum(  x[ij2n(j,i),v] for v in vrange for j in range(N) if v > 0) == 4)
    model.Add(sum(v*x[ij2n(j,i),v] for v in vrange for j in range(N)) == 20)

# no 2x2 filled subgrid
for i,j in product(range(N-1), range(N-1)):
    mat22 = (x[ij2n(i  ,j),v] + x[ij2n(i  ,j+1),v] +
             x[ij2n(i+1,j),v] + x[ij2n(i+1,j+1),v]
             for v in vrange if v > 0)
    model.Add(sum(mat22) <= 3)

#### Correct views on $x$
Consider the requirement that when looking from the left of row $i$, the first value that appears must be some specified $v'$.

Equivalently, we require for each column $j$ in row $i$ that
1. If $v'$ has not appeared yet (for all $j' <j$), then no other $v>0$ can appear, i.e.
    * $\sum_{j'} \sum_{v>0} x_{n, v} = 0$, where $n$ is the one-dimensional index coressponding to $(i,j')$.
2. If $v'$ has already appeared, then there are no constraints on $v$.

This can be done using the big-$M$, where we can essentially nullify a constraint once a desired state occurs, i.e.
$$
\sum_{j'} \sum_{v>0} x_{n, v} \leq M \sum_{j'} x_{n, v'}
$$

If we look from the right of a row, then we consider all $j' > j$. Looking from top and below, we simply consider in terms of $(i', j)$.

In [ ]:
def addViewConstraint(self, view, side):
    if not view: # i.e. empty constraint
        return
    x = self.x
    N = self.N
    vrange = self.vrange

    M = 10 # penalty for if target v appeared already (6 should be sufficient)

    # left and top views just see head of row/col
    if side == "l" or side == "t":
        jprange = lambda j: range(j)
    # right and bot views just see tail of row/col
    elif side == "r" or side == "b":
        jprange = lambda j: range(j+1, N)

    # top and bottom look at columns, i.e. transposed
    if side == "t" or side == "b":
        ij2n = lambda i, j: ij2n(j,i)
    elif side == "l" or side == "r":
        ij2n = ij2n

    for it, vt in view:
        for j in range(N):
            v_before  = sum(x[ij2n(it,jp),v ] for jp in jprange(j) for v in vrange if v not in (0, vt))
            vt_before = sum(x[ij2n(it,jp),vt] for jp in jprange(j))
            self.model.Add(v_before <= M*vt_before)

## top-bot-left-right views correct
addViewConstraint(views['top_view'], "t")
addViewConstraint(views['bot_view'], "b")
addViewConstraint(views['lft_view'], "l")
addViewConstraint(views['rgt_view'], "r")

#### Connected components
* If either cell $(n_1)$ or $(n_2)$ are empty, then $y_{n_2,n_2}$ should be 0, i.e.
    $$
    2 y_{n_1,n_2} \leq \sum_{v > 0} x_{n_1, v} + x_{n_2, v}
    $$
    If $(n_1)$ and $(n_2)$ are both filled, this constraint alone does not force $y_{n_1,n_2}$ to 1, it merely **allows** it.
* Given the sparsity constraint (i.e. no filled $2\times 2$ submatrices), the total number of edges is simply one less than the total number of values filled in, i.e. $28-1=27$.
    $$
    \sum_{n_1, n_2} y_{n_1,n_2} = 27
    $$

With these two constraints working together, all 27 edges which are allowed to exist will exist.

In [ ]:
for i,j in erange: 
    model.Add(2*y[nl,nr] <= sum(x[nl,v] + x[nr,v] for v in vrange if v != 0))
    model.Add(2*y[nu,nd] <= sum(x[nu,v] + x[nd,v] for v in vrange if v != 0))

# connected component (total edges = (nnz-1))
nnz = sum(v for v in vrange) # number of elements = one 1, two 2, ... seven 7's, should be 28

model.Add(
    sum(
        y[ij2n(i,j),ij2n(i, j+1)] + 
        y[ij2n(j,i),ij2n(j+1, i)]
        for i,j in erange)
    == nnz-1)

#### Instance constraints
* Cell $(n)$ contains value $(v)$:
    * $x_{n,v}$ = 1

In [ ]:
# prescribed values
for (i,j), val in vals_list:
    model.Add(x[ij2n(i,j),val] == 1)


#### Solve

In [ ]:
# SOLVE
solver = cp_model.CpSolver()
status = solver.Solve(model)